In [1]:
import findspark
findspark.init("/home/ozan/spark")

import os
os.environ["PYSPARK_SUBMIT_ARGS"]= '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.1 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark= SparkSession.builder \
.master("local[4]") \
.appName("sparkproje") \
.getOrCreate()

In [2]:
def uzunluk(start,stop):
    valuex=(stop[0]-start[0])**2
    valuey=(stop[1]-start[1])**2
    return round(sqrt(valuex+valuey),2)

# Step 1

In [3]:
#Kafka bağlanan Dataframe'i oluştruma
df_step11= spark \
.readStream \
.format("kafka") \
.option("kafka.bootstrap.servers","localhost:9092") \
.option("subscribe","gps") \
.load()

df_step12 = df_step11.selectExpr("CAST(value AS STRING)")

#query= df_step12.writeStream.format("console").outputMode("append").start()
#query.stop()

# Step 2

In [4]:
#Şemayı oluşturma ve akan dataya yerleştirme
schema=StructType([
    StructField("gps_id",StringType()),
    StructField("start",ArrayType(FloatType())),
    StructField("stop",ArrayType(FloatType()))
    
])

df_step2=df_step12.withColumn("value",from_json("value",schema).alias("gps"))

#query= df_step2.select(col("value.gps_id"),col("value.start"),col("value.stop"),).writeStream.format("console").outputMode("append").start()
#query.stop()

# Step 3

In [5]:
#Mesafeleri hesaplamak için uzunluk fonksiyonunu kullanma
df_step3=df_step2.select(
    col("value.gps_id"),
    col("value.start"),
    col("value.stop"),
    uzunluk(col("value.start"),col("value.stop")).alias("distanse"))

#query= df_step3.select("gps_id","start","stop","distanse").writeStream.format("console").outputMode("append").start()
#query.stop()

# Step 4

In [6]:
#Gruop By, Sum ve sort işlemlerini yapma
df_step4=df_step3.groupBy("gps_id").\
agg(round(sum("distanse"),2).alias("total_distanse")).\
sort(desc("total_distanse"))

#query= df_step4.writeStream.format("console").outputMode("complete").start()
#query.stop()

# Step 5

In [7]:
#Kullanıcıların bulunduğu csv dosyası ile join işlemini yapma
userdf=spark.read.format("csv")\
.option("sep",";")\
.option("header","true")\
.option("encoding","ISO-8859-9")\
.load("users.csv")


df_step5=df_step4.join(userdf,df_step4.gps_id==userdf.gps_id)\
.select(df_step4.gps_id,df_step4.total_distanse,userdf.user_id,userdf.first_name)


query= df_step5.writeStream.format("console").outputMode("complete").start()
#query.stop()

In [8]:
#Query durdurma
query.stop()